# custom_aisearch_agent.py - ELI5 Walkthrough
This notebook reconstructs a proof-of-concept workflow that pairs Azure AI Search retrieval with an Agent Framework pipeline.
It mirrors the sample structure used throughout these ELI5 notebooks so you can explore each stage of the orchestration step by step.


## Big Picture
Use Azure AI Search to pull grounded context, hand the enriched payload to an Azure AI Agent, and surface the
final answer as the workflow output.


## Key Ingredients
- `AzureSearchRetriever` wraps the Azure AI Search SDK and keeps the index hydrated with sample data.
- `RetrieveAzureSearchContext` calls the retriever from inside a workflow executor node.
- `AzureAIAgentClient` spins up hosted Azure AI agents that can live inside the workflow graph.
- `WorkflowBuilder` connects the retrieval and answer stages without hand-rolled orchestration code.


## Workflow Diagram
```mermaid
flowchart LR
    Q(["User Query"]) --> R[["Azure AI Search Executor"]]
    R --> A[["Azure AI Agent"]]
    A --> O(["Grounded Answer"])
```


### Step 1: Imports, configuration, and scenario overview
Load environment variables first, then pull in the Azure AI Search SDK and Agent Framework helpers used throughout the sample.


In [12]:
# Copyright (c) Microsoft. All rights reserved.
from dotenv import load_dotenv
load_dotenv()

import asyncio
import os
from collections.abc import Awaitable, Callable
from contextlib import AsyncExitStack
from typing import Any, Dict, Iterable, List, Optional

from agent_framework import (
    AgentRunUpdateEvent,
    Executor,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,
    handler,
)
from agent_framework.azure import AzureAIAgentClient
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import ResourceExistsError, ResourceNotFoundError
from azure.identity.aio import AzureCliCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndex,
    SearchFieldDataType,
    SearchableField,
    SimpleField,
)

"""
Scenario summary:
- Use Azure AI Search to fetch relevant snippets for a user question.
- Feed those snippets into a hosted Azure AI agent that drafts the final answer.
- Demonstrate how execution tasks and agent tasks cooperate inside WorkflowBuilder.

Environment prerequisites (document these values in `.env.example`):
- `AZURE_SEARCH_ENDPOINT`
- `AZURE_SEARCH_API_KEY`
- `AZURE_SEARCH_INDEX_NAME`
- `AZURE_AI_PROJECT_NAME`, `AZURE_AI_AGENT_NAME` (or any additional settings required by Azure AI Agents service)

Optional: set `SEED_SEARCH_INDEX=true` to load a small in-memory data set for quick demos.
"""


'\nScenario summary:\n- Use Azure AI Search to fetch relevant snippets for a user question.\n- Feed those snippets into a hosted Azure AI agent that drafts the final answer.\n- Demonstrate how execution tasks and agent tasks cooperate inside WorkflowBuilder.\n\nEnvironment prerequisites (document these values in `.env.example`):\n- `AZURE_SEARCH_ENDPOINT`\n- `AZURE_SEARCH_API_KEY`\n- `AZURE_SEARCH_INDEX_NAME`\n- `AZURE_AI_PROJECT_NAME`, `AZURE_AI_AGENT_NAME` (or any additional settings required by Azure AI Agents service)\n\nOptional: set `SEED_SEARCH_INDEX=true` to load a small in-memory data set for quick demos.\n'

### Step 2: Wrap Azure AI Search in a lightweight helper
The helper below owns the Search clients, ensures the index schema exists, optionally seeds sample documents, and exposes a
`get_retrieval_context` method that returns a newline-joined string of document excerpts.


In [13]:
class AzureSearchRetriever:
    """Utility that manages index setup and fetches snippets using Azure AI Search."""

    def __init__(self, endpoint: str, api_key: str, index_name: str) -> None:
        credential = AzureKeyCredential(api_key)
        self.index_name = index_name
        self.index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
        self.search_client = SearchClient(endpoint=endpoint, index_name=index_name, credential=credential)

    def ensure_index(self) -> None:
        """Create the search index if it does not already exist."""
        fields = [
            SimpleField(name="id", type=SearchFieldDataType.String, key=True),
            SearchableField(name="content", type=SearchFieldDataType.String, searchable=True),
        ]
        index = SearchIndex(name=self.index_name, fields=fields)
        try:
            self.index_client.get_index(self.index_name)
            return
        except ResourceNotFoundError:
            pass
        try:
            self.index_client.create_index(index)
        except ResourceExistsError:
            return

    def seed_documents(self, documents: Iterable[Dict[str, Any]]) -> None:
        """Upload sample documents to make the demo self-contained.

        Skip if the iterable is empty to avoid unnecessary service calls.
        """
        docs = list(documents)
        if not docs:
            return
        self.search_client.upload_documents(documents=docs)

    def get_retrieval_context(self, query: str) -> str:
        """Run a simple full-text search and concatenate the snippets."""
        results = self.search_client.search(query)
        context_strings: List[str] = []
        for result in results:
            context_strings.append(f"Document: {result['content']}")
        return "".join(context_strings) if context_strings else "No results found"


SAMPLE_DOCUMENTS: List[Dict[str, Any]] = [
    {"id": "1", "content": "Contoso Banking waives monthly fees for savings accounts with automated deposits."},
    {"id": "2", "content": "Customers can reach Contoso support 24/7 via chat, phone, or the mobile app."},
    {"id": "3", "content": "The rewards credit card includes travel insurance and zero foreign transaction fees."},
]


### Step 3: Define the retrieval executor and prepare an Azure AI agent factory
One executor asks Azure AI Search for context, and the hosted agent consumes a formatted string that includes both the context
and the original question.


In [20]:
class RetrieveAzureSearchContext(Executor):
    """Fetch search results for the inbound query and forward context downstream."""

    def __init__(self, retriever: AzureSearchRetriever) -> None:
        super().__init__(id="azure_ai_search_lookup")
        self.retriever = retriever

    @handler
    async def handle(self, query: str, ctx: WorkflowContext[str, str]) -> None:
        context = self.retriever.get_retrieval_context(query)
        prompt = (
            "You are given CONTEXT and a QUESTION. Use only the CONTEXT to answer."
            "If the CONTEXT says 'No results found', respond that the answer is unavailable."
            f"""
CONTEXT: {context}
QUESTION: {query}
ANSWER:"""
        )
        await ctx.send_message(prompt)


async def create_azure_ai_agent() -> tuple[Callable[..., Awaitable[Any]], Callable[[], Awaitable[None]]]:
    """Helper method to create an Azure AI agent factory and a close function."""
    stack = AsyncExitStack()
    cred = await stack.enter_async_context(AzureCliCredential())
    client = await stack.enter_async_context(AzureAIAgentClient(async_credential=cred))

    async def agent(**kwargs: Any) -> Any:
        return await stack.enter_async_context(client.create_agent(**kwargs))

    async def close() -> None:
        await stack.aclose()

    return agent, close


def build_search_workflow(
    retriever: AzureSearchRetriever,
    answer_agent: Any,
) -> Any:
    """Construct the workflow that runs retrieval then routes to the Azure AI agent."""
    retrieve = RetrieveAzureSearchContext(retriever)
    return (
        WorkflowBuilder()
        .set_start_executor(retrieve)
        .add_edge(retrieve, answer_agent)
        .build()
    )


### Step 4: Bootstrap everything, optionally seed the index, and stream the workflow output
The cell below wires the helper objects together. It uses `run_stream()` so you can watch the agent respond token-by-token.


In [21]:
async def main() -> None:
    """Instantiate clients, build the workflow, and execute a sample query."""
    search_endpoint = os.environ.get("AZURE_SEARCH_ENDPOINT")
    search_api_key = os.environ.get("AZURE_SEARCH_API_KEY")
    index_name = os.environ.get("AZURE_SEARCH_INDEX_NAME")

    if not all([search_endpoint, search_api_key, index_name]):
        raise RuntimeError("Missing Azure AI Search configuration. Check .env settings.")

    agent_factory, close = await create_azure_ai_agent()
    try:
        retriever = AzureSearchRetriever(
            endpoint=search_endpoint,
            api_key=search_api_key,
            index_name=index_name,
        )
        retriever.ensure_index()

        if os.environ.get("SEED_SEARCH_INDEX", "false").lower() in {"1", "true", "yes"}:
            retriever.seed_documents(SAMPLE_DOCUMENTS)

        answer_agent = await agent_factory(
            name="BankingSearchResponder",
            instructions=(
                "You answer customer banking questions and only cite information present in the CONTEXT you receive."
                "Respond in 2-3 sentences and avoid inventing details."
            ),
        )

        workflow = build_search_workflow(retriever, answer_agent)

        last_executor_id: Optional[str] = None
        events = workflow.run_stream("What perks does the rewards credit card include?")
        async for event in events:
            if isinstance(event, AgentRunUpdateEvent):
                eid = event.executor_id
                if eid != last_executor_id:
                    if last_executor_id is not None:
                        print()
                    print(f"{eid}:", end=" ", flush=True)
                    last_executor_id = eid
                print(event.data, end="", flush=True)
            elif isinstance(event, WorkflowOutputEvent):
                print("===== Final output =====")
                print(event.data)
    finally:
        await close()


!az login --tenant 16b3c013-d300-468d-ac64-7eda0820b6d3

In [25]:
import asyncio

# Helper for notebooks vs. scripts
loop = asyncio.get_event_loop()
if loop.is_running():
    await main()
else:
    asyncio.run(main())


BankingSearchResponder: The rewards credit card includes travel insurance and zero foreign transaction fees.===== Final output =====
The rewards credit card includes travel insurance and zero foreign transaction fees.
